Step 1: Install and Import Required Libraries

In [ ]:
!pip install --quiet google-cloud-bigquery

In [ ]:
from google.cloud import bigquery


Step 2: Set Project and Dataset Configuration

In [ ]:
PROJECT_ID = "qwiklabs-gcp-01-79d56652b122"
DATASET_ID = "fraud_lab"

client = bigquery.Client(project=PROJECT_ID)

Step 3: Import Emergency Response Data into BigQuery

In [ ]:
raw_table_id = f"{PROJECT_ID}.{DATASET_ID}.emergency_calls_response_times"

job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
)

source_uri = (
    "gs://labs.roitraining.com/data-to-ai-workshop/"
    "emergency_calls_response_times.csv"
)

load_job = client.load_table_from_uri(
    source_uri,
    raw_table_id,
    job_config=job_config,
)

load_job.result()

print("Emergency response data loaded into BigQuery.")


Emergency response data loaded into BigQuery.


Step 4: Study the Data

In [ ]:
client.query(f"""
select *
from `{raw_table_id}`
limit 20
""").to_dataframe()


,call_id,call_timestamp,call_type,location,weather_condition,day_of_week,time_of_day,traffic_level,distance_to_station,units_available,response_time
0,35957,2023-01-01 00:05:53+00:00,Fire,Highland,Rainy,Sunday,0,High,21.45,3,23.41
1,20832,2023-01-01 00:20:47+00:00,Fire,Oakmont,Rainy,Sunday,0,High,22.29,6,20.11
2,27949,2023-01-01 00:33:27+00:00,Fire,Riverside,Windy,Sunday,0,High,17.19,14,19.75
3,20199,2023-01-01 00:48:29+00:00,Fire,Riverside,Windy,Sunday,0,High,17.39,14,20.76
4,46938,2023-01-01 00:50:44+00:00,Rescue,Brookfield,Sunny,Sunday,0,High,22.50,14,22.37
5,17582,2023-01-01 02:28:50+00:00,Rescue,Downtown,Snowy,Sunday,2,High,25.15,6,28.48
6,21624,2023-01-01 02:44:06+00:00,Rescue,Oakmont,Snowy,Sunday,2,High,3.95,9,19.30
7,36793,2023-01-01 02:53:54+00:00,Fire,Riverside,Sunny,Sunday,2,High,5.87,10,10.72
8,41350,2023-01-01 03:52:33+00:00,Police,Greenfield,Windy,Sunday,3,High,6.66,5,20.55
9,32092,2023-01-01 04:09:23+00:00,Police,Maplewood,Snowy,Sunday,4,High,15.50,13,22.98


In [ ]:
client.query(f"""
select
  column_name,
  data_type
from `{PROJECT_ID}.{DATASET_ID}.INFORMATION_SCHEMA.COLUMNS`
where table_name = 'emergency_calls_response_times'
order by ordinal_position
""").to_dataframe()


,column_name,data_type
0,call_id,INT64
1,call_timestamp,TIMESTAMP
2,call_type,STRING
3,location,STRING
4,weather_condition,STRING
5,day_of_week,STRING
6,time_of_day,INT64
7,traffic_level,STRING
8,distance_to_station,FLOAT64
9,units_available,INT64


Step 5: Create a BigQuery ML Model

In [ ]:
model_id = f"{PROJECT_ID}.{DATASET_ID}.emergency_response_model"

sql = f"""
create or replace model `{model_id}`
options (
  model_type = 'linear_reg',
  input_label_cols = ['response_time']
) as
select
  call_type,
  location,
  weather_condition,
  day_of_week,
  time_of_day,
  traffic_level,
  distance_to_station,
  units_available,
  response_time
from `{raw_table_id}`
where response_time is not null
"""

client.query(sql).result()
print("BigQuery ML model created.")


BigQuery ML model created.


Step 6: Evaluate the Model

In [ ]:
client.query(f"""
select *
from ML.EVALUATE(MODEL `{model_id}`)
""").to_dataframe()


,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,1.742121,4.770663,0.014883,1.474353,0.829923,0.829955


Step 7: Make a Prediction Using Synthetic Data

In [20]:
client.query(f"""
select *
from ML.PREDICT(
  model `{model_id}`,
  (
    select
      'Medical' AS call_type,
      'Downtown' AS location,
      'Clear' AS weather_condition,
      'Monday' AS day_of_week,
      14 AS time_of_day,
      'High' AS traffic_level,
      3.2 AS distance_to_station,
      5 AS units_available
  )
)
""").to_dataframe()


,predicted_response_time,call_type,location,weather_condition,day_of_week,time_of_day,traffic_level,distance_to_station,units_available
0,16.973959,Medical,Downtown,Clear,Monday,14,High,3.2,5
